# KARPHY'S CNN 
The notebook is used to experiment with video classification using 

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [149]:
# tf.enable_eager_execution()

## Reading in data
loading "ucf101" dataset from tensorflow. More info here URL: https://www.crcv.ucf.edu/data/UCF101.php

In [2]:
ucf101_dataset, ucf101_info = tfds.load(name="ucf101", with_info=True)
ucf101_train , ucf101_test = ucf101_dataset["train"], ucf101_dataset["test"]
# assert isinstance(ucf101_info, tf.data.DatasetInfo)
print(type(ucf101_info))
assert isinstance(ucf101_train, tf.data.Dataset)
assert isinstance(ucf101_test, tf.data.Dataset)

print('number of training examples:', ucf101_info.splits["train"].num_examples)
print('number of test examples:', ucf101_info.splits["test"].num_examples)
print('number of labels:', ucf101_info.features["label"].num_classes)


<class 'tensorflow_datasets.core.dataset_info.DatasetInfo'>
number of training examples: 9537
number of test examples: 3783
number of labels: 101


Getting a small amount of data

In [52]:
first_1_percent_train_split = tfds.Split.TRAIN.subsplit(tfds.percent[:1])
first_1_percent_test_split = tfds.Split.TEST.subsplit(tfds.percent[-1:])
combined_first_1_percent_split = first_1_percent_test_split + first_1_percent_train_split
one_percent_dataset, info = tfds.load("ucf101", split=combined_first_1_percent_split, with_info=True )

print('number of training examples:', info.splits["train"].num_examples)
print('number of test examples:', info.splits["test"].num_examples)
print('number of labels:', info.features["label"].num_classes)
print('number of samples', one_percent_dataset)


number of training examples: 9537
number of test examples: 3783
number of labels: 101
number of samples <_OptionsDataset shapes: {video: (None, 256, 256, 3), label: ()}, types: {video: tf.uint8, label: tf.int64}>


converting dataset to numpy array

### Looking into the dataset
looking into the dataset

In [55]:
print(one_percent_dataset)
# get_label_name = metadata.feature['label'].int2str
for d in one_percent_dataset.take(2):
    print(d["label"])
#     print(d["video"].shape)
#     print('single frame',d["video"][0].shape)
#     print('single frame',d["video"].numpy()[0].reshape(1,256,256,3).shape)

<_OptionsDataset shapes: {video: (None, 256, 256, 3), label: ()}, types: {video: tf.uint8, label: tf.int64}>
tf.Tensor(97, shape=(), dtype=int64)
tf.Tensor(25, shape=(), dtype=int64)


### Formating data
* normalizing pixels 
* rescaling image to 170 x 170 x 3 as per pare 
* normalize number of frames

In [19]:
IMG_SIZE = 170

def format_videos(dataset):
    dataset["video"] = tf.cast(dataset["video"], tf.float32)
    dataset["video"] = (dataset["video"]/255)
    dataset["video"] = tf.image.resize(dataset["video"], (IMG_SIZE,IMG_SIZE))
    return dataset
    

In [34]:
def select_frame(dataset):
    dataset["video"] = dataset["video"][0:10]
    return dataset 
    

In [27]:
def convert_to_tuple(dataset):
    x = dataset["video"]
    y = dataset["label"]
    return x,y 

changing resizing image and 

In [61]:
ucf101_train_data = ucf101_train.map(format_videos)
ucf101_train_single_frame_data = ucf101_train_data.map(select_frame)
train_tp = ucf101_train_single_frame_data.map(convert_to_tuple)
for d in ucf101_train_single_frame_data.take(1):
    print(d["label"])
    print(d["video"].shape)  
for x, y in train_tp.take(1):
    print(x.shape)   
#     for i in d["video"]
#     for i in range(4):
#         plt.figure()
#         plt.imshow(d["video"][i])

tf.Tensor(96, shape=(), dtype=int64)
(10, 170, 170, 3)
(10, 170, 170, 3)


Displaying images in video last image

loading test dataset

In [35]:
first_1_test_percent = tfds.Split.TEST.subsplit(tfds.percent[:1])
test_dataset = tfds.load("ucf101", split=first_1_test_percent)
test_dataset = test_dataset.map(format_videos)
for d in test_dataset.take(2):
    print(d["label"].numpy())
    print(d["video"].shape)

76
(333, 170, 170, 3)
88
(201, 170, 170, 3)


##

## Single Frame model

Creating single frame model from paper

Creating custom layer for local response normalization

In [33]:
class MyLRNLayer(tf.keras.layers.Layer):
    def __init__(self,depth_radius=5,bias=1,alpha=1,beta=0.5, **kwargs):
#         self.output_dim = output_dim
        self.depth_radius = depth_radius
        self.bias = bias
        self.alpha = alpha
        self.beta = beta
        super(MyLRNLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape),
                                      initializer='uniform',
                                      trainable=False)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return tf.nn.local_response_normalization(x,self.depth_radius,self.bias,self.alpha,self.beta)

    def compute_output_shape(self, input_shape):
        return input_shape

In [39]:
layer = MyLRNLayer(10)
# print(layer(tf.zeros([10, 5])))
# print(layer.trainable_variables)

In [36]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3)),
    
  MyLRNLayer(),  
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu'),
    
  MyLRNLayer(), 
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
    
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
    
  tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu'),
    
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
    
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(101, activation='softmax')
])

ValueError: Cannot convert a partially known TensorShape to a Tensor: (None, 54, 54, 96)

In [43]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 23, 23, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 256)       1024      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 21, 21, 384)       885120    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 384)      

In [44]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [46]:
# train_np = tfds.as_numpy(train_dataset)
# train_dataset = train_dataset.shuffle(1000).batch(128).prefetch(10)
# ucf101_train_single_frame_dataset = ucf101_train_single_frame_data.shuffle(1000).batch(128).prefetch(10)
# train_np = tfds.as_numpy(ucf101_train_single_frame_dataset)
# print(type(train_np))
# print(train_np.shape)
# print(type(train_dataset.shuffle(1000)))
model.fit(train_tp, 
      epochs=3,
      verbose=1)

Epoch 1/15
9537/9537 [==============================] - 2345s 246ms/step - loss: 15.9575 - accuracy: 0.0100
Epoch 2/15
9537/9537 [==============================] - 2337s 245ms/step - loss: 15.9575 - accuracy: 0.0100
Epoch 3/15
4603/9537 [=============>................] - ETA: 21:44 - loss: 15.9570 - accuracy: 0.0100

KeyboardInterrupt: 

# Improving the single frame model by using batch normalization 

In [ ]:
model = tf.keras.models.Sequential([
#      stride value invalid
  tf.keras.layers.Conv2D(96, (11,11), strides=3 , activation='relu', input_shape=(170, 170, 3)),
#     tf.keras.layers.Conv2D(96, (11,11),  activation='relu', input_shape=(170, 170, 3)),
  tf.keras.layers.BatchNormalization(),
#   tf.nn.local_response_normalization(depth_radius=5,bias=2,alpha=0.0001,beta=0.5),  
#     tf.keras.layers.LRN2D(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (5,5), strides=1, activation='relu'),
    
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
    
  tf.keras.layers.Conv2D(384, (3,3), strides=1, activation='relu'),
  tf.keras.layers.Conv2D(256, (3,3), strides=1, activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(101, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.fit(train_tp, 
      epochs=3,
      verbose=1)

In [238]:
data, info = tfds.load("ucf101", with_info=True)
train_data, test_data = data['train'], data['test']
assert isinstance(train_data, tf.data.Dataset)
print(info.features['label'].num_classes)
print(info.splits['train'].num_examples)

101
9537


In [239]:
history = model.fit_generator(train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])

NameError: name 'train_generator' is not defined

In [7]:
np.random.rand?

In [216]:
tf.keras.layers?

In [220]:
model.fit?

In [224]:
train_dataset["label"]

TypeError: 'MapDataset' object is not subscriptable

In [7]:
tf.keras.layers.?

Object `tf.keras.layers.LRN2D` not found.
